In [2]:
# ロジスティックネットワークの最適設計

import numpy as np
import pandas as pd

from itertools import product
from pulp import LpVariable, lpSum, value
from ortoolpy import model_min, addvars, addvals

df_tc = pd.read_csv('../data/trans_cost.csv', index_col='工場')
df_demand = pd.read_csv('../data/demand.csv')
df_supply = pd.read_csv('../data/supply.csv')

df_tc.head()

,F1,F2,F3,F4
工場,,,,
W1,10,10,11,27
W2,18,21,12,14
W3,15,12,14,12


In [3]:
df_demand.head()


,F1,F2,F3,F4
0,28,29,31,25


In [4]:
df_supply.head()


,W1,W2,W3
0,35,41,42


In [5]:
np.random.seed(1)
nw = len(df_tc.index)
nf = len(df_tc.columns)
pr = list(product(range(nw), range(nf)))

pr

[(0, 0),
 (0, 1),
 (0, 2),
 (0, 3),
 (1, 0),
 (1, 1),
 (1, 2),
 (1, 3),
 (2, 0),
 (2, 1),
 (2, 2),
 (2, 3)]

In [6]:
m1 = model_min()
v1 = {(i, j): LpVariable('v%d_%d'%(i, j), lowBound=0) for i, j in pr}
m1 += lpSum(df_tc.iloc[i][j] * v1[i, j] for i, j in pr)
for i in range(nw):
    m1 += lpSum(v1[i, j] for j in range(nf)) <= df_supply.iloc[0][i]
for j in range(nf):
    m1 += lpSum(v1[i, j] for i in range(nw)) >= df_demand.iloc[0][j]
m1.solve()


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/omitsuhashi/.local/share/virtualenvs/knock-MS8y58kW/lib/python3.9/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/y2/4bd1s8gj0tddrbwc4bb9pcmr0000gn/T/d987b488cb8c40f2a9dbc37be6874d8c-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/y2/4bd1s8gj0tddrbwc4bb9pcmr0000gn/T/d987b488cb8c40f2a9dbc37be6874d8c-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 12 COLUMNS
At line 49 RHS
At line 57 BOUNDS
At line 58 ENDATA
Problem MODEL has 7 rows, 12 columns and 24 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 7 (0) rows, 12 (0) columns and 24 (0) elements
0  Obj 0 Primal inf 113 (4)
7  Obj 1296
Optimal - objective value 1296
Optimal objective 1296 - 7 iterations time 0.002
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallcloc

1

In [7]:
df_tr_sol = df_tc.copy()
total_cost = 0
for k, x in v1.items():
    i, j = k[0], k[1]
    df_tr_sol.iloc[i][j] = value(x)
    total_cost += df_tc.iloc[i][j] * value(x)

print(df_tr_sol)
print('total cost: ', total_cost)


    F1  F2  F3  F4
工場                
W1  28   7   0   0
W2   0   0  31   5
W3   0  22   0  20
total cost:  1296.0


In [8]:
v1

{(0, 0): v0_0,
 (0, 1): v0_1,
 (0, 2): v0_2,
 (0, 3): v0_3,
 (1, 0): v1_0,
 (1, 1): v1_1,
 (1, 2): v1_2,
 (1, 3): v1_3,
 (2, 0): v2_0,
 (2, 1): v2_1,
 (2, 2): v2_2,
 (2, 3): v2_3}